# Import modules and load data

In [1]:
!pip install -U torch
!pip install medialpy
!pip install tensorflow
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 5.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 74.3 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 60.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 112.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 71.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 10.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 18.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 37.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 26.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 18.5 MB/s eta 0:00:0000:0100:01
     ━━━━━

In [2]:
#autoload modules in notebook
%load_ext autoreload
%autoreload 2  

In [2]:
import pandas as pd
import tensorflow as tf
import wandb
from transformers import DistilBertTokenizer, TFDistilBertModel
from sklearn.model_selection import train_test_split

import CLIP_model
import CLIP_data_load
import train_utils
import random
import numpy as np
import torch

2023-09-05 11:08:43.288800: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 11:08:44.010769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ale/miniconda3/envs/tf2/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [30]:
# Hyperparameters
from hyperparameters import *

print(f'Using random seed {SEED}...')
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
tf.random.set_seed(SEED)


bert_model_name = 'distilbert-base-uncased' #bert-base-multilingual-uncased

model_name = 'test_clip_base'

# Paths
all_images_path = 'dataset/resized_train/'
project_location = '.'
all_captions_path = f'{project_location}/dataset/caption_prediction_train.csv'  
all_concept_ids_path = f'{project_location}/dataset/concept_detection_train.csv'
all_concepts_path = f'{project_location}/dataset/concepts.csv'

zip_dataset_location = f'{project_location}/dataset/resized_train.zip'
dataset_extract_location = "/"
model_loc = f'{project_location}/weights/{model_name}.h5'

Using random seed 116...


In [31]:
run = wandb.init(
    project="clip",
    entity="calonca",
    name=model_name,
    config={
        "batch_size": batch_size,
        "latent_dim_common": latent_dim_common,
        "latent_dim_text": latent_dim_text,
        "latent_dim_imgs": latent_dim_imgs,
        "img_shape": img_shape,
        "captions_input_shape": captions_input_shape,
        "concepts_input_shape": concepts_input_shape,
        "model_name": model_name,
        "bert_model_name": bert_model_name,
        "lr_image_encoder": lr_image_encoder,
        "lr_text_encoder": lr_text_encoder,
        "lr_projector": lr_projector,
        "projector_dropout_rate": projector_dropout_rate,
        "zs_num_classes": zs_num_classes,
    },
)

epoch,▁▁
loss,██▁▁
best_epoch,0
best_loss,4.38904
epoch,0
loss,4.38904


# Data preparation

In [102]:
#Merging all dataframes toghether into ID, caption, cuis, concepts
captionsDF = pd.read_csv(all_captions_path, sep='\t')
concept_id_df = pd.read_csv(all_concept_ids_path, sep='\t')
concept_df = pd.read_csv(all_concepts_path, sep=',')

captions_concepts_df = CLIP_data_load.preprocess_captions_concepts(captionsDF, concept_id_df, concept_df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [103]:
captions_concepts_df= captions_concepts_df[:1000]

In [104]:
train_val, test = train_test_split(captions_concepts_df, test_size = test_percentage, shuffle = True, random_state = SEED)
train, val = train_test_split(train_val, test_size = validation_percentage, shuffle = True, random_state = SEED)

In [105]:
tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name)

trainList = CLIP_data_load.paths_captions_concepts_emb_list(
    train,
    all_images_path,
    tokenizer=tokenizer,
    max_len_captions=captions_input_shape[0],
    max_len_concepts=concepts_input_shape[0],
    remove_images_threshold=0
)
valList = CLIP_data_load.paths_captions_concepts_emb_list(
    val,
    all_images_path,
    tokenizer=tokenizer,
    max_len_captions=captions_input_shape[0],
    max_len_concepts=concepts_input_shape[0],
    remove_images_threshold=0
)

100%|██████████| 120/120 [00:00<00:00, 690.09it/s]


In [106]:
len(valList)

120

In [107]:
train_gen = CLIP_data_load.FusionGenerator(
    channels_first=False,
    preprocessing_function=None,
    data=trainList,
    batch_size=batch_size,
    out_shape=img_shape[0:2],
    shuffle=True,
    model_version='base'
)
val_gen = CLIP_data_load.FusionGenerator(
    channels_first=False,
    preprocessing_function=None,
    data=valList,
    batch_size=batch_size,
    out_shape=img_shape[0:2],
    shuffle=False,
    model_version='base'
)

### Model creation

In [108]:
fen_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    pooling='avg',
    input_shape = img_shape
)

bert_model = TFDistilBertModel.from_pretrained(bert_model_name)

model = CLIP_model.get_clip_model(
    image_input_shape = img_shape,
    text_input_shape = captions_input_shape,
    text_encoder = bert_model,
    image_encoder = fen_model,
    latent_dim_imgs = latent_dim_imgs,
    latent_dim_text = latent_dim_text,
    latent_dim_common = latent_dim_common,
    train_bert = True,
    lr_image_encoder=lr_image_encoder,
    lr_text_encoder=lr_text_encoder,
    lr_projector=lr_projector,
    projector_dropout_rate=projector_dropout_rate,
    loss=train_utils.clip_loss
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [109]:
model.summary()

Model: "clip_base_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_35 (InputLayer)       [(None, 128)]                0         []                            
                                                                                                  
 input_36 (InputLayer)       [(None, 128)]                0         []                            
                                                                                                  
 input_34 (InputLayer)       [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 tf_distil_bert_model_8 (TF  TFBaseModelOutput(last_hid   6636288   ['input_35[0][0]',            
 DistilBertModel)            den_state=(None, 128, 768)   0          'input_36[0][0]']  

### Model training

In [110]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)

In [111]:
def embed_txt(batch_ids,batch_att):
  image_zeros = tf.zeros([batch_ids.shape[0],*img_shape])
  return embed(image_zeros,batch_ids,batch_att)[0]

def embed(batch_img,batch_ids,batch_att):
    e_txt,e_img = model.predict_step((batch_img,batch_ids,batch_att))
    return e_txt.numpy(),e_img.numpy()

tk_labels, _ , one_hot_labels,  = train_utils.build_zero_shot_metric(
    test,
    tokenizer,
    concepts_input_shape[0],
    captions_input_shape[0],
    preceding_caption="An image of ",
    num_classes=zs_num_classes,
)

emb_fun = lambda : embed_txt(tk_labels['input_ids'], tk_labels['attention_mask'])

zero_shot_callback = train_utils.ZeroShotSingleLabelCallBack(
    model=model,
    gen=val_gen,
    zs_labels=one_hot_labels,
    logger=wandb,
    emb_fun=emb_fun
)   

In [112]:
len(train_gen)

8

In [ ]:
history = model.fit(
    x=train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[
        train_utils.wandb_callback_fun(val_gen),
        zero_shot_callback,
        train_utils.early_stopping,
        train_utils.checkpoint_fun(f"{project_location}/weights/{model_name}.h5"),
    ],
    workers=4,
)

Epoch 1/20
8/8 [==============================] - 72s 2s/step - loss: 4.3549 - val_loss: 4.3764 - zero_shot_accuracy: 0.0250
Epoch 2/20
8/8 [==============================] - 15s 2s/step - loss: 4.1527 - val_loss: 4.3465 - zero_shot_accuracy: 0.0250
Epoch 3/20
8/8 [==============================] - 14s 2s/step - loss: 4.0110 - val_loss: 4.4000 - zero_shot_accuracy: 0.0125
Epoch 4/20
8/8 [==============================] - 16s 2s/step - loss: 3.7731 - val_loss: 4.4933 - zero_shot_accuracy: 0.0125
Epoch 5/20
8/8 [==============================] - 13s 2s/step - loss: 3.5836 - val_loss: 4.3381 - zero_shot_accuracy: 0.0000e+00
Epoch 6/20
8/8 [==============================] - 14s 2s/step - loss: 3.4366 - val_loss: 4.3237 - zero_shot_accuracy: 0.0000e+00
Epoch 7/20
8/8 [==============================] - 15s 2s/step - loss: 3.3168 - val_loss: 4.3730 - zero_shot_accuracy: 0.0250
Epoch 8/20
8/8 [==============================] - 14s 2s/step - loss: 3.2076 - val_loss: 4.2990 - zero_shot_accuracy:

15/15 [==============================] - 16s 1s/step - loss: 1.2840 - val_loss: 2.0582 - zero_shot_accuracy: 0.0000e+00
Epoch 6/20
2/2 [==============================] - 1s 247ms/steposs: 1.

Epoch 6: val_loss did not improve from 2.05815
15/15 [==============================] - 16s 1s/step - loss: 1.0127 - val_loss: 2.2317 - zero_shot_accuracy: 0.0000e+00
Epoch 7/20
2/2 [==============================] - 1s 258ms/steposs: 1.

Epoch 7: val_loss did not improve from 2.05815
15/15 [==============================] - 16s 1s/step - loss: 1.4846 - val_loss: 2.2712 - zero_shot_accuracy: 0.0625
Epoch 8/20
2/2 [==============================] - 1s 246ms/steposs: 1.

Epoch 8: val_loss improved from 2.05815 to 2.00344, saving model to ./weights/test_clip_base.h5


/home/ale/miniconda3/envs/tf2/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15/15 [==============================] - 16s 1s/step - loss: 1.2091 - val_loss: 2.0034 - zero_shot_accuracy: 0.0000e+00
Epoch 9/20
2/2 [==============================] - 1s 241ms/steposs: 0.

Epoch 9: val_loss did not improve from 2.00344
15/15 [==============================] - 15s 1s/step - loss: 0.9131 - val_loss: 2.3950 - zero_shot_accuracy: 0.0000e+00
Epoch 10/20
2/2 [==============================] - 1s 288ms/steposs: 0.

Epoch 10: val_loss did not improve from 2.00344
15/15 [==============================] - 16s 1s/step - loss: 0.7623 - val_loss: 2.3311 - zero_shot_accuracy: 0.0000e+00
Epoch 11/20
2/2 [==============================] - 1s 241ms/steposs: 1.

Epoch 11: val_loss did not improve from 2.00344
15/15 [==============================] - 16s 1s/step - loss: 1.0073 - val_loss: 2.3673 - zero_shot_accuracy: 0.0625
Epoch 12/20
2/2 [==============================] - 1s 249ms/steposs: 1.

Epoch 12: val_loss did not improve from 2.00344
15/15 [==============================] - 1

### Model save

In [18]:
artifact = wandb.Artifact(model_name, type='model')
artifact.add_file(f"{project_location}/weights/{model_name}.h5")
run.log_artifact(artifact)
run.finish()